The first step is importing necessary modules. Long term, the idea is this will reside in the ArcGIS Python API, so instead of all the stuff in the cell below, getting started will be as easy as... 


```
from arcgis.dm import Country
from arcgis.dm.util import get_countries
```

Currently, while figuring out how to make all this work, it is not quite this easy, and there is an error message since I am adding some functionality onto the Spatially Enabled DataFrame, the `spatial` attribute for the Pandas DataFrame.

In [1]:
from pathlib import Path
import sys

import arcpy

project_parent = Path('./').absolute().parent

# import the project package from the project package path
# ideally will be imported using 'from arcgis import dm'
sys.path.append(str(project_parent/'src'))
import dm

# load the "autoreload" extension so as src code is changed, the changes are picked up in the dataframe
%load_ext autoreload
%autoreload 2

D:\projects\demographic-modeling-module\src\dm\_modify_geoaccessor.py:31: UserWarning: registration of accessor <class 'dm._modify_geoaccessor.GeoAccessorIO'> under name 'spatial' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexistingattribute with the same name.
  class GeoAccessorIO(GeoAccessor):


# Enrich in a Country

Enrichment is performed against data for a given country. Consequently, the first step in enrichment is creating a demographic modeling Country object instance. Creating a Country object instance requires knowing the three letter identifier for the country, and this can be discovered by using the `get_countries` function included with teh dmeographic modeling utilities.

In [2]:
dm.util.get_countries()

,geographic_level,country,year
0,USA_ESRI_2019,USA,2019


Since only the data for the United States is installed on this machine, this is the only country listed. In this dataframe, the value from the `country` column, `USA`, is what will be used to create a Country object instance. Additionally, when instantiating the Country object instance, if ArcGIS Pro with Business Analyst is available, `local` will be used by default, but can also be explicitly specified using the source parameter. Also, if an `arcgis.gis.GIS` with enrichment is passed into the `source` parameter, this can be used as well (_online not yet implemented_).

In [3]:
usa = dm.Country('USA', source='local')

usa

<class: Country - USA (local)>

Within each country, there are a multitude of heirarchial standard geographic resolutions available. Analysis in the United States usually identifies an area of interest by a single Core Based Statistical Area (CBSA) or Designated Market Area (DMA), and then uses a smaller geographic resolution for analysis, such as Block Groups. Hence, it is useful to be able to introspectively discover what is available. This is possible by accessing the `geographies` property of the country object instance.

The `geographies` property is lazily retrieved, but once retrieved, is cached as a property of the country object instance. Any subsequent retreivals do not require re-retrieving this information from the sources.

In [4]:
usa.geographies

,geo_name,geo_alias,col_id,col_name,feature_class_path
0,block_groups,Block Groups,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
1,census_tracts,Census Tracts,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
2,cities_and_towns_places,Cities and Towns (Places),ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
3,zip_codes,ZIP Codes,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
4,county_subdivisions,County Subdivisions,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
5,counties,Counties,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
6,cbsas,CBSAs,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
7,congressional_districts,Congressional Districts,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
8,dmas,DMAs,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
9,states,States,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...


# Get Enrichment Geographies in an Area of Interest

Getting an area of interest from the country is the programatic equivalent of, "_I want the Seattle CBSA_," even though the _real_ name is the `Seattle-Tacoma-Bellevue, WA Metroplitan Statistical Area`. Referencing the DataFrame displayed above, we can use the `geo_name` on the country object to `get` the `seattle` CBSA as a Spatially Enabled DataFrame.

In [5]:
cbsa_df = usa.cbsas.get('seattle')

cbsa_df

,ID,NAME,SHAPE
0,42660,"Seattle-Tacoma-Bellevue, WA Metropolitan Stati...","{""rings"": [[[-13651055.7226, 5968866.240900002..."


Since a Spatially Enabled DataFrame, we can use the Web Map Widget to quickly check what the retrieved data looks like - see if it is what we want.

In [6]:
webmap = cbsa_df.spatial.plot()
webmap.basemap = 'gray-vector'
webmap

MapView(layout=Layout(height='400px', width='100%'))

While most of the time we know what the geographic level of resolution is, frequently we do not care; we just want the lowest and most accurate level available. For this reason, we also can retrive geography by simply referencing the index from the geographies DataFrame above, getting the lowest level, block groups in the United States, by referencing index zero. Further, for the sake of code efficiency, using functing chaining, we can retrieve teh lowest level of geography, block groups, in the study area, the Seattle CBSA.

In [7]:
#lvl_df = usa.cbsas.get('seattle').block_groups.get()
lvl_df = usa.cbsas.get('seattle').level(0).get()

lvl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2474 entries, 0 to 2473
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   ID      2474 non-null   object  
 1   NAME    2474 non-null   object  
 2   SHAPE   2474 non-null   geometry
dtypes: geometry(1), object(2)
memory usage: 58.1+ KB


The resultant dataframe is fairly simple, a table identical to the CBSA table above with three columns, ID, NAME, and the geometry in the last, the SHAPE column.

In [8]:
lvl_df.head()

,ID,NAME,SHAPE
0,530530714071,530530714.071,"{""rings"": [[[-13618997.0451, 5953796.150899999..."
1,530530714072,530530714.072,"{""rings"": [[[-13621890.9066, 5953114.284599997..."
2,530530714073,530530714.073,"{""rings"": [[[-13622599.9004, 5953136.347999997..."
3,530530714112,530530714.112,"{""rings"": [[[-13627506.4183, 5953782.585000001..."
4,530530729061,530530729.061,"{""rings"": [[[-13654973.1668, 5957970.395400003..."


Again, since a Spatially Enabled DataFrame, we can again visually check what the results look like to ensure this looks like what we are expecting.

In [9]:
# checking what the results look like as a simple map
webmap02 = lvl_df.spatial.plot()
webmap02.basemap = 'gray-vector'
webmap02

MapView(layout=Layout(height='400px', width='100%'))

# Enrich the Spatially Enabled DataFrame

Before performing enrichment, we must first discover what variables are available (_over 9,000_), and select the variables we want to use for our analysis.

In [10]:
usa.enrich_variables.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9078 entries, 0 to 9077
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               9078 non-null   object
 1   alias              9078 non-null   object
 2   type               9078 non-null   object
 3   vintage            9078 non-null   object
 4   data_collection    9078 non-null   object
 5   enrich_str         9078 non-null   object
 6   enrich_field_name  9078 non-null   object
dtypes: object(7)
memory usage: 496.6+ KB


In [11]:
usa.enrich_variables.head()

,name,alias,type,vintage,data_collection,enrich_str,enrich_field_name
0,AGE0_CY,2019 Population Age <1,COUNT,2019,1yearincrements,1yearincrements.AGE0_CY,F1yearincrements_AGE0_CY
1,AGE1_CY,2019 Population Age 1,COUNT,2019,1yearincrements,1yearincrements.AGE1_CY,F1yearincrements_AGE1_CY
2,AGE2_CY,2019 Population Age 2,COUNT,2019,1yearincrements,1yearincrements.AGE2_CY,F1yearincrements_AGE2_CY
3,AGE3_CY,2019 Population Age 3,COUNT,2019,1yearincrements,1yearincrements.AGE3_CY,F1yearincrements_AGE3_CY
4,AGE4_CY,2019 Population Age 4,COUNT,2019,1yearincrements,1yearincrements.AGE4_CY,F1yearincrements_AGE4_CY


Since a Pandas DataFrame, filtering methods can be easily used to discover and select variables. In this case, this is a common set of variables I use to start analysis, Key Facts for the most recent year.

In [12]:
usa.enrich_variables[(usa.enrich_variables.data_collection == 'KeyUSFacts') & (usa.enrich_variables.vintage == '2019')]

,name,alias,type,vintage,data_collection,enrich_str,enrich_field_name
6450,TOTPOP_CY,2019 Total Population,COUNT,2019,KeyUSFacts,KeyUSFacts.TOTPOP_CY,KeyUSFacts_TOTPOP_CY
6452,GQPOP_CY,2019 Population in Group Quarters,COUNT,2019,KeyUSFacts,KeyUSFacts.GQPOP_CY,KeyUSFacts_GQPOP_CY
6453,DIVINDX_CY,2019 Diversity Index,COUNT,2019,KeyUSFacts,KeyUSFacts.DIVINDX_CY,KeyUSFacts_DIVINDX_CY
6456,TOTHH_CY,2019 Total Households,COUNT,2019,KeyUSFacts,KeyUSFacts.TOTHH_CY,KeyUSFacts_TOTHH_CY
6458,AVGHHSZ_CY,2019 Average Household Size,COUNT,2019,KeyUSFacts,KeyUSFacts.AVGHHSZ_CY,KeyUSFacts_AVGHHSZ_CY
6459,MEDHINC_CY,2019 Median Household Income,CURRENCY,2019,KeyUSFacts,KeyUSFacts.MEDHINC_CY,KeyUSFacts_MEDHINC_CY
6461,AVGHINC_CY,2019 Average Household Income,CURRENCY,2019,KeyUSFacts,KeyUSFacts.AVGHINC_CY,KeyUSFacts_AVGHINC_CY
6463,PCI_CY,2019 Per Capita Income,CURRENCY,2019,KeyUSFacts,KeyUSFacts.PCI_CY,KeyUSFacts_PCI_CY
6467,TOTHU_CY,2019 Total Housing Units,COUNT,2019,KeyUSFacts,KeyUSFacts.TOTHU_CY,KeyUSFacts_TOTHU_CY
6469,OWNER_CY,2019 Owner Occupied HUs,COUNT,2019,KeyUSFacts,KeyUSFacts.OWNER_CY,KeyUSFacts_OWNER_CY


Using this filter, the strings referencing the enrich variables can be collected and saved into a variable.

In [13]:
enrich_vars = usa.enrich_variables[(usa.enrich_variables.data_collection == 'KeyUSFacts') & (usa.enrich_variables.vintage == '2019')].enrich_str

enrich_vars

6450     KeyUSFacts.TOTPOP_CY
6452      KeyUSFacts.GQPOP_CY
6453    KeyUSFacts.DIVINDX_CY
6456      KeyUSFacts.TOTHH_CY
6458    KeyUSFacts.AVGHHSZ_CY
6459    KeyUSFacts.MEDHINC_CY
6461    KeyUSFacts.AVGHINC_CY
6463        KeyUSFacts.PCI_CY
6467      KeyUSFacts.TOTHU_CY
6469      KeyUSFacts.OWNER_CY
6471     KeyUSFacts.RENTER_CY
6473     KeyUSFacts.VACANT_CY
6475     KeyUSFacts.MEDVAL_CY
6477     KeyUSFacts.AVGVAL_CY
6479    KeyUSFacts.POPGRW10CY
6480     KeyUSFacts.HHGRW10CY
6481    KeyUSFacts.FAMGRW10CY
6487       KeyUSFacts.DPOP_CY
6488    KeyUSFacts.DPOPWRK_CY
6489    KeyUSFacts.DPOPRES_CY
Name: enrich_str, dtype: object

Now, the previoiusly retrieved data can be enriched using the above referenced variables.

In [14]:
bg_enrich_df = lvl_df.spatial.enrich(enrich_vars)

bg_enrich_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2474 entries, 0 to 2473
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   ID                     2474 non-null   object  
 1   NAME                   2474 non-null   object  
 2   KeyUSFacts_TOTPOP_CY   2474 non-null   float64 
 3   KeyUSFacts_GQPOP_CY    2474 non-null   float64 
 4   KeyUSFacts_DIVINDX_CY  2474 non-null   float64 
 5   KeyUSFacts_TOTHH_CY    2474 non-null   float64 
 6   KeyUSFacts_AVGHHSZ_CY  2474 non-null   float64 
 7   KeyUSFacts_MEDHINC_CY  2474 non-null   float64 
 8   KeyUSFacts_AVGHINC_CY  2474 non-null   float64 
 9   KeyUSFacts_PCI_CY      2474 non-null   float64 
 10  KeyUSFacts_TOTHU_CY    2474 non-null   float64 
 11  KeyUSFacts_OWNER_CY    2474 non-null   float64 
 12  KeyUSFacts_RENTER_CY   2474 non-null   float64 
 13  KeyUSFacts_VACANT_CY   2474 non-null   float64 
 14  KeyUSFacts_MEDVAL_CY   2474 non-null   f

In [15]:
bg_enrich_df.head()

,ID,NAME,KeyUSFacts_TOTPOP_CY,KeyUSFacts_GQPOP_CY,KeyUSFacts_DIVINDX_CY,KeyUSFacts_TOTHH_CY,KeyUSFacts_AVGHHSZ_CY,KeyUSFacts_MEDHINC_CY,KeyUSFacts_AVGHINC_CY,KeyUSFacts_PCI_CY,...,KeyUSFacts_VACANT_CY,KeyUSFacts_MEDVAL_CY,KeyUSFacts_AVGVAL_CY,KeyUSFacts_POPGRW10CY,KeyUSFacts_HHGRW10CY,KeyUSFacts_FAMGRW10CY,KeyUSFacts_DPOP_CY,KeyUSFacts_DPOPWRK_CY,KeyUSFacts_DPOPRES_CY,SHAPE
0,530530714071,530530714.071,1653.0,0.0,53.5,508.0,3.25,93624.0,102731.0,31571.0,...,13.0,298837.0,402310.0,1.31,1.15,1.08,1082.0,159.0,923.0,"{""rings"": [[[-13618997.0451, 5953796.150899999..."
1,530530714072,530530714.072,1530.0,6.0,59.4,490.0,3.11,71613.0,92289.0,29563.0,...,36.0,262662.0,265588.0,-0.14,-0.22,-0.27,901.0,50.0,851.0,"{""rings"": [[[-13621890.9066, 5953114.284599997..."
2,530530714073,530530714.073,1657.0,0.0,67.3,520.0,3.19,76880.0,95900.0,30095.0,...,16.0,206471.0,226697.0,0.57,0.49,0.45,1085.0,364.0,721.0,"{""rings"": [[[-13622599.9004, 5953136.347999997..."
3,530530714112,530530714.112,1343.0,0.0,71.4,473.0,2.84,55898.0,68939.0,24280.0,...,27.0,232576.0,234836.0,2.87,2.78,2.48,1294.0,498.0,796.0,"{""rings"": [[[-13627506.4183, 5953782.585000001..."
4,530530729061,530530729.061,2934.0,2913.0,57.2,7.0,3.00,0.0,0.0,24059.0,...,0.0,0.0,0.0,8.82,0.00,0.00,3065.0,3062.0,3.0,"{""rings"": [[[-13654973.1668, 5957970.395400003..."


For the sake of coding efficiency, function chaning can be used to perform all the above steps in one line to accomplish the same task to get the final result.

In [25]:
%%time
usa = dm.Country('USA')
lvl_enrich_df = usa.cbsas.get('seattle').level(0).get().spatial.enrich(enrich_vars)

lvl_enrich_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2474 entries, 0 to 2473
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   ID                     2474 non-null   object  
 1   NAME                   2474 non-null   object  
 2   KeyUSFacts_TOTPOP_CY   2474 non-null   float64 
 3   KeyUSFacts_GQPOP_CY    2474 non-null   float64 
 4   KeyUSFacts_DIVINDX_CY  2474 non-null   float64 
 5   KeyUSFacts_TOTHH_CY    2474 non-null   float64 
 6   KeyUSFacts_AVGHHSZ_CY  2474 non-null   float64 
 7   KeyUSFacts_MEDHINC_CY  2474 non-null   float64 
 8   KeyUSFacts_AVGHINC_CY  2474 non-null   float64 
 9   KeyUSFacts_PCI_CY      2474 non-null   float64 
 10  KeyUSFacts_TOTHU_CY    2474 non-null   float64 
 11  KeyUSFacts_OWNER_CY    2474 non-null   float64 
 12  KeyUSFacts_RENTER_CY   2474 non-null   float64 
 13  KeyUSFacts_VACANT_CY   2474 non-null   float64 
 14  KeyUSFacts_MEDVAL_CY   2474 non-null   f

In [27]:
lvl_enrich_df.head(10)

,ID,NAME,KeyUSFacts_TOTPOP_CY,KeyUSFacts_GQPOP_CY,KeyUSFacts_DIVINDX_CY,KeyUSFacts_TOTHH_CY,KeyUSFacts_AVGHHSZ_CY,KeyUSFacts_MEDHINC_CY,KeyUSFacts_AVGHINC_CY,KeyUSFacts_PCI_CY,...,KeyUSFacts_VACANT_CY,KeyUSFacts_MEDVAL_CY,KeyUSFacts_AVGVAL_CY,KeyUSFacts_POPGRW10CY,KeyUSFacts_HHGRW10CY,KeyUSFacts_FAMGRW10CY,KeyUSFacts_DPOP_CY,KeyUSFacts_DPOPWRK_CY,KeyUSFacts_DPOPRES_CY,SHAPE
0,530530714071,530530714.071,1653.0,0.0,53.5,508.0,3.25,93624.0,102731.0,31571.0,...,13.0,298837.0,402310.0,1.31,1.15,1.08,1082.0,159.0,923.0,"{""rings"": [[[-13618997.0451, 5953796.150899999..."
1,530530714072,530530714.072,1530.0,6.0,59.4,490.0,3.11,71613.0,92289.0,29563.0,...,36.0,262662.0,265588.0,-0.14,-0.22,-0.27,901.0,50.0,851.0,"{""rings"": [[[-13621890.9066, 5953114.284599997..."
2,530530714073,530530714.073,1657.0,0.0,67.3,520.0,3.19,76880.0,95900.0,30095.0,...,16.0,206471.0,226697.0,0.57,0.49,0.45,1085.0,364.0,721.0,"{""rings"": [[[-13622599.9004, 5953136.347999997..."
3,530530714112,530530714.112,1343.0,0.0,71.4,473.0,2.84,55898.0,68939.0,24280.0,...,27.0,232576.0,234836.0,2.87,2.78,2.48,1294.0,498.0,796.0,"{""rings"": [[[-13627506.4183, 5953782.585000001..."
4,530530729061,530530729.061,2934.0,2913.0,57.2,7.0,3.00,0.0,0.0,24059.0,...,0.0,0.0,0.0,8.82,0.00,0.00,3065.0,3062.0,3.0,"{""rings"": [[[-13654973.1668, 5957970.395400003..."
5,530530730011,530530730.011,1282.0,0.0,36.5,448.0,2.86,69019.0,85202.0,29774.0,...,39.0,196032.0,215575.0,0.14,0.07,-0.03,987.0,233.0,754.0,"{""rings"": [[[-13642145.2653, 5936675.103100002..."
6,530530730012,530530730.012,1002.0,64.0,30.9,340.0,2.76,86200.0,100866.0,34317.0,...,13.0,379921.0,587284.0,0.92,0.90,0.81,649.0,146.0,503.0,"{""rings"": [[[-13642388.3871, 5939707.134900004..."
7,530530730013,530530730.013,1332.0,0.0,42.5,470.0,2.83,89554.0,86851.0,30646.0,...,31.0,343307.0,368041.0,0.77,0.69,0.61,734.0,34.0,700.0,"{""rings"": [[[-13639074.6285, 5941447.145999998..."
8,530530730014,530530730.014,1810.0,0.0,43.1,662.0,2.73,68273.0,82828.0,30294.0,...,31.0,491080.0,730502.0,3.07,2.99,2.90,1098.0,38.0,1060.0,"{""rings"": [[[-13633815.3391, 5941422.011100002..."
9,530530730015,530530730.015,1410.0,0.0,41.5,537.0,2.63,69198.0,86446.0,32923.0,...,37.0,224706.0,258396.0,0.99,0.91,0.79,1086.0,311.0,775.0,"{""rings"": [[[-13641028.9534, 5944010.630999997..."


This can be saved to a variety formats including Parquet, CSV and an ArcGIS Feature Class.

In [34]:
can = dm.Country('CAN')

In [35]:
can.geographies

,geo_name,geo_alias,col_id,col_name,feature_class_path
0,disseminationareas,DisseminationAreas,ID,NAME,D:\ba_data\can\Data\Demographic Data\can_esri_...
1,censustracts,CensusTracts,ID,NAME,D:\ba_data\can\Data\Demographic Data\can_esri_...
2,fsas,FSAs,ID,NAME,D:\ba_data\can\Data\Demographic Data\can_esri_...
3,censussubdivisions,CensusSubdivisions,ID,NAME,D:\ba_data\can\Data\Demographic Data\can_esri_...
4,feds,FEDs,ID,NAME,D:\ba_data\can\Data\Demographic Data\can_esri_...
5,cmacas,CMACAs,ID,NAME,D:\ba_data\can\Data\Demographic Data\can_esri_...
6,censusdivisions,CensusDivisions,ID,NAME,D:\ba_data\can\Data\Demographic Data\can_esri_...
7,provinceterritories,ProvinceTerritories,ID,NAME,D:\ba_data\can\Data\Demographic Data\can_esri_...
8,country,Country,ID,NAME,D:\ba_data\can\Data\Demographic Data\can_esri_...


In [36]:
can.enrich_variables

,name,alias,type,vintage,data_collection,enrich_str,enrich_field_name
0,A16AITFNAT,2016 First Nations Single Ident,COUNT,2016,AboriginalIdentity,AboriginalIdentity.A16AITFNAT,AboriginalIdentity_A16AITFNAT
1,A16AITIDT,2016 Aboriginal Identity,COUNT,2016,AboriginalIdentity,AboriginalIdentity.A16AITIDT,AboriginalIdentity_A16AITIDT
2,A16AITIDX,2016 Aboriginal Identities,COUNT,2016,AboriginalIdentity,AboriginalIdentity.A16AITIDX,AboriginalIdentity_A16AITIDX
3,A16AITINUK,2016 Inuk Single Identity,COUNT,2016,AboriginalIdentity,AboriginalIdentity.A16AITINUK,AboriginalIdentity_A16AITINUK
4,A16AITMETI,2016 Metis Single Identity,COUNT,2016,AboriginalIdentity,AboriginalIdentity.A16AITMETI,AboriginalIdentity_A16AITMETI
...,...,...,...,...,...,...,...
2568,P0YVISKOR,2029 VM: Korean,COUNT,2029,VisibleMinorityStatus,VisibleMinorityStatus.P0YVISKOR,VisibleMinorityStatus_P0YVISKOR
2569,P0YVISJAPA,2029 VM: Japanese,COUNT,2029,VisibleMinorityStatus,VisibleMinorityStatus.P0YVISJAPA,VisibleMinorityStatus_P0YVISJAPA
2570,P0YVISOVM,2029 VM: All Oth VM,COUNT,2029,VisibleMinorityStatus,VisibleMinorityStatus.P0YVISOVM,VisibleMinorityStatus_P0YVISOVM
2571,P0YVISMVM,2029 VM: Multiple VM,COUNT,2029,VisibleMinorityStatus,VisibleMinorityStatus.P0YVISMVM,VisibleMinorityStatus_P0YVISMVM


In [17]:
lvl_enrich_df.spatial.to_parquet('../data/interim/seattl_bg_enriched.parquet')

When saving to an ArcGIS Feature Class, it is useful to also set the aliases after saving the Feature Class. This can be done by looking up the aliases from the Country object instance.

In [ ]:
out_fc = r'../data/interim/interim.gdb/seattle_bg_enriched'

lvl_enrich_df.spatial.to_featureclass(out_fc)

for fld_nm in lvl_enrich_df.columns:
    
    fld_df = usa.enrich_variables[usa.enrich_variables.enrich_field_name.str.contains(fld_nm)]
    
    if len(fld_df.index):
        
        arcpy.management.AlterField(
            in_table=out_fc,
            field=fld_nm,
            new_field_alias=fld_df.iloc[0]['alias']
        )

In [19]:
out_df = lvl_enrich_df

In [20]:
out_df.spatial.set_geometry('SHAPE')

In [ ]:
out_df.spatial.validate()

In [22]:
bg_df = dm.Country('USA').cbsas.get('seattle').level(0).get()
bg_df

,ID,NAME,SHAPE
0,530530714071,530530714.071,"{""rings"": [[[-13618997.0451, 5953796.150899999..."
1,530530714072,530530714.072,"{""rings"": [[[-13621890.9066, 5953114.284599997..."
2,530530714073,530530714.073,"{""rings"": [[[-13622599.9004, 5953136.347999997..."
3,530530714112,530530714.112,"{""rings"": [[[-13627506.4183, 5953782.585000001..."
4,530530729061,530530729.061,"{""rings"": [[[-13654973.1668, 5957970.395400003..."
...,...,...,...
2469,530610538013,530610538.013,"{""rings"": [[[-13511805.726300001, 6123144.5534..."
2470,530610536023,530610536.023,"{""rings"": [[[-13575742.8551, 6123969.329099998..."
2471,530610536024,530610536.024,"{""rings"": [[[-13574886.8082, 6127115.140100002..."
2472,530610537002,530610537.002,"{""rings"": [[[-13563583.0931, 6156537.565200001..."


In [23]:
bg_df.spatial.validate()

True